In [ ]:
import numpy as np
import keras
import pickle
from keras.utils.io_utils import HDF5Matrix
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.models import load_model
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight
import itertools
from functions import encode, decode, plot_confusion_matrix, plot_metrics, plot_images, class_weights_from_labels, f1, plot_f1
%matplotlib inline

In [ ]:
keras.__version__

In [ ]:
train_path = '/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/annotated images balanced/train'
valid_path = '/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/annotated images balanced/valid'
test_path = '/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/annotated images balanced/test'

In [ ]:
with open('/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/dataset/classes_reduced.txt', "rb") as fp:
    classes = pickle.load(fp)

In [ ]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path, target_size=(224, 224), 
                                                         classes=classes, batch_size=32)

valid_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(valid_path, target_size=(224, 224), 
                                                         classes=classes, batch_size=32)

test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path, target_size=(224, 224), 
                                                        classes=classes, batch_size=100)

In [ ]:
vgg16_model = load_model('/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/models/mlb_classifier.h5', custom_objects={'f1': f1})

In [ ]:
vgg16_model.summary()

In [ ]:
batch_size = 32
learning_rate = 1e-4
steps_per_epoch = 13500 // batch_size #to go through all examples
val_steps = 2700 // batch_size #32*10 = 320 examples for validation --> do not use all examples --> reduce time of computation
epochs = 20

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1', min_delta=1e-4, patience=5, verbose=1, 
                                               mode='max')
filepath = '/Users/diegoeduardo/Desktop/Image Gravity Project/Coral Reef Data desk/models/mlb_classifier.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_f1', verbose=1, save_best_only=True, 
                                             save_weights_only=False, mode='max', period=1)

In [ ]:
history = vgg16_model.fit_generator(train_batches, steps_per_epoch=steps_per_epoch, 
                                    epochs=epochs, validation_data=valid_batches, 
                                    validation_steps=val_steps,
                                    callbacks=[checkpoint, early_stopping], verbose=1)